In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import XGBModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [3]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}


In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
def objective(trial):
    num_target_lags = trial.suggest_int('num_target_lags', 5, 24)
    target_lags_list = [-i for i in range(1, num_target_lags + 1)]
    output_chunk_length = trial.suggest_int("output_chunk_length", 1, 24)
    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600, step=50),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.1),
        'gamma': trial.suggest_float('gamma', 0, 0.3, step=0.05),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 0.5, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 0.5, log=True),
        'n_jobs': -1
    }

    n = int(len(df) * 0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nStarting Trial {trial.number}")
    print(f"Hyperparameters: {trial.params}")
    print("\nTraining the model...")
    print(f"Train set: {len(train_fold)} samples")
    print(f"Validation set: {len(val_fold)} samples")

    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/"
    _model_name = "model_optuna_temp"
    os.makedirs(_work_dir, exist_ok=True)

    model = XGBModel(
        lags=target_lags_list,
        output_chunk_length=output_chunk_length,
        add_encoders=encoders,
        **xgb_params
    )

    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=False,
    )

    forecasts = model.historical_forecasts(
        val_scaled,
        forecast_horizon=output_chunk_length,
        stride=1,
        retrain=False,
        verbose=False
    )

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(val_scaled)
    metrics = {}
    print("Starting time series verification for consistency...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nPerformance metrics:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_mae_val = mae(val_scaled, forecasts)
    print(f"The MAE for this fold was {overall_mae_val}")

    trial_dict = {
        "trial_number": trial.number,
        "overall_mae_val": overall_mae_val,
        "output_chunk_length": output_chunk_length,
        "xgb_params": xgb_params,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return overall_mae_val if not np.isnan(overall_mae_val) else float("inf")


In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 11
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-05-27 10:54:59,372] A new study created in memory with name: no-name-900a06d5-b888-49e3-bc53-db8dc3aad324
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.



Starting Trial 0
Hyperparameters: {'num_target_lags': 13, 'output_chunk_length': 17, 'n_estimators': 550, 'max_depth': 5, 'learning_rate': 0.125856139636599, 'min_child_weight': 5, 'subsample': 0.7, 'colsample_bytree': 1.0, 'gamma': 0.2, 'reg_alpha': 0.00014455126712679928, 'reg_lambda': 0.0017534131927373943}

Training the model...
Train set: 6647 samples
Validation set: 1662 samples


[I 2025-05-27 10:55:25,276] Trial 0 finished with value: 0.13509872168495973 and parameters: {'num_target_lags': 13, 'output_chunk_length': 17, 'n_estimators': 550, 'max_depth': 5, 'learning_rate': 0.125856139636599, 'min_child_weight': 5, 'subsample': 0.7, 'colsample_bytree': 1.0, 'gamma': 0.2, 'reg_alpha': 0.00014455126712679928, 'reg_lambda': 0.0017534131927373943}. Best is trial 0 with value: 0.13509872168495973.
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


Starting time series verification for consistency...

Performance metrics:
                                     MSE        RMSE         MAE         R2  \
Temperature                    11.024118    3.320259    2.688736   0.637926   
Precipitation_accumulated      65.503924    8.093449    7.865879 -18.085965   
Humidity                       99.874825    9.993739    8.110559   0.318410   
Wind_Speed_kmh                  1.303359    1.141648    0.905619   0.034318   
Soil_Moisture              201917.202452  449.351981  388.806389  -2.529073   
Soil_Temperature                8.650031    2.941093    2.453218   0.463377   
Wind_Dir_Sin                    0.647735    0.804820    0.709065  -0.176605   
Wind_Dir_Cos                    0.439930    0.663272    0.575073  -0.199790   

                                SMAPE  
Temperature                 40.148189  
Precipitation_accumulated    1.707197  
Humidity                     9.552924  
Wind_Speed_kmh              74.810107  
Soil_Moisture

[I 2025-05-27 10:56:15,875] Trial 1 finished with value: 0.13619483404911095 and parameters: {'num_target_lags': 24, 'output_chunk_length': 7, 'n_estimators': 150, 'max_depth': 8, 'learning_rate': 0.013094316669732781, 'min_child_weight': 6, 'subsample': 1.0, 'colsample_bytree': 1.0, 'gamma': 0.15000000000000002, 'reg_alpha': 0.0029844990648053697, 'reg_lambda': 0.00850717668386233}. Best is trial 0 with value: 0.13509872168495973.
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


...

Starting time series verification for consistency...

Performance metrics:
                                   MSE       RMSE        MAE         R2  \
Temperature                   2.713323   1.647217   1.278233   0.911181   
Precipitation_accumulated    56.349904   7.506657   7.243625 -15.102903   
Humidity                     11.018194   3.319366   2.643376   0.924874   
Wind_Speed_kmh                0.544676   0.738022   0.549801   0.598242   
Soil_Moisture              1102.890797  33.209800  19.572021   0.981503   
Soil_Temperature              0.851000   0.922497   0.629232   0.947480   
Wind_Dir_Sin                  0.361675   0.601394   0.486967   0.340811   
Wind_Dir_Cos                  0.310110   0.556875   0.448797   0.162365   

                                SMAPE  
Temperature                 26.256900  
Precipitation_accumulated    1.571129  
Humidity                     3.078015  
Wind_Speed_kmh              57.244547  
Soil_Moisture                0.231050  
Soil_Temp

[I 2025-05-27 11:15:34,949] Trial 75 finished with value: 0.09693789020817435 and parameters: {'num_target_lags': 16, 'output_chunk_length': 2, 'n_estimators': 550, 'max_depth': 4, 'learning_rate': 0.022211013120711377, 'min_child_weight': 7, 'subsample': 0.9, 'colsample_bytree': 0.8, 'gamma': 0.15000000000000002, 'reg_alpha': 7.034480328712667e-05, 'reg_lambda': 0.02407262829527278}. Best is trial 74 with value: 0.07975567767844809.
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


Starting time series verification for consistency...

Performance metrics:
                                   MSE       RMSE        MAE         R2  \
Temperature                   5.804707   2.409296   1.908355   0.810031   
Precipitation_accumulated    62.859127   7.928375   7.579132 -16.989172   
Humidity                     26.759289   5.172938   4.177151   0.817535   
Wind_Speed_kmh                0.921589   0.959994   0.730440   0.320633   
Soil_Moisture              1869.997915  43.243473  23.559342   0.968534   
Soil_Temperature              2.276381   1.508768   1.064149   0.859515   
Wind_Dir_Sin                  0.462866   0.680343   0.585642   0.155854   
Wind_Dir_Cos                  0.338572   0.581869   0.487231   0.085277   

                                SMAPE  
Temperature                 33.235126  
Precipitation_accumulated    1.644873  
Humidity                     4.887414  
Wind_Speed_kmh              67.419556  
Soil_Moisture                0.275631  
Soil_Temp

[I 2025-05-27 11:15:39,266] Trial 76 finished with value: 0.09796510330819941 and parameters: {'num_target_lags': 13, 'output_chunk_length': 2, 'n_estimators': 550, 'max_depth': 4, 'learning_rate': 0.02041575558178987, 'min_child_weight': 7, 'subsample': 0.9, 'colsample_bytree': 0.8, 'gamma': 0.2, 'reg_alpha': 4.4908569830781666e-05, 'reg_lambda': 0.013678243258206633}. Best is trial 74 with value: 0.07975567767844809.


Starting time series verification for consistency...

Performance metrics:
                                   MSE       RMSE        MAE         R2  \
Temperature                   6.729173   2.594065   2.065699   0.779543   
Precipitation_accumulated    64.449423   8.028040   7.801910 -17.363290   
Humidity                     30.869360   5.556020   4.601342   0.789500   
Wind_Speed_kmh                0.917419   0.957820   0.726061   0.322694   
Soil_Moisture              1994.620963  44.661180  24.950230   0.966767   
Soil_Temperature              1.794216   1.339483   0.950720   0.889251   
Wind_Dir_Sin                  0.456465   0.675622   0.582949   0.168891   
Wind_Dir_Cos                  0.344540   0.586975   0.492226   0.069209   

                                SMAPE  
Temperature                 35.050704  
Precipitation_accumulated    1.693256  
Humidity                     5.381518  
Wind_Speed_kmh              67.366906  
Soil_Moisture                0.291850  
Soil_Temp

In [8]:
print(f"Best Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best Value (Minimum): 0.07975567767844809
Best Parameters: {'num_target_lags': 16, 'output_chunk_length': 1, 'n_estimators': 550, 'max_depth': 4, 'learning_rate': 0.021776422794897058, 'min_child_weight': 7, 'subsample': 0.9, 'colsample_bytree': 0.8, 'gamma': 0.1, 'reg_alpha': 4.745332363644534e-05, 'reg_lambda': 0.012349873114991963}


In [9]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")


Best results saved in optuna_iteration_metrics/best_trial.json
